## Testing OpenAI's new Responses API and Agents SDK

OpenAI announced on March 11th, 2025 new Responses API and Agents SDK (https://openai.com/index/new-tools-for-building-agents/) 

This notebook aims to test these new products

In [137]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

from pydantic import BaseModel, Field
import json

from IPython.display import display, Markdown

#prev openai version: 1.59.8
#new openai version: 1.66.2

### 1. Responses API

Syntax

In [8]:
MODEL_NAME = "gpt-4o"
instruction = "Write a one-sentence bedtime story about a unicorn."

### ChatCompletions

completion = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": instruction
        }
    ]
)

### Responses API

response = client.responses.create(
    model=MODEL_NAME,
    temperature=0.0,
    input=[
        {
            "role": "user",
            "content": instruction
        }
    ]
)

In [9]:
print(f'ChatCompletion response: \n{completion.choices[0].message.content}')
print('---' * 20)
print(f'Responses response: \n{response.output_text}')

ChatCompletion response: 
In a moonlit meadow, a gentle unicorn named Luna sprinkled stardust with her shimmering horn, lulling the entire forest into a peaceful, enchanted slumber.
------------------------------------------------------------
Responses response: 
Under the shimmering moonlight, Luna the unicorn gently trotted through the enchanted forest, her mane sparkling with stardust, as she whispered dreams of magic and wonder to all the sleeping creatures.


In [18]:
response.output_text == response.output[0].content[0].text

True

Explore new Responses output object

In [37]:
MODEL_NAME = "o3-mini"

system_message = "You are a witty joke writer. You have no political correctness filter. You're number one goal is to tell jokes without being afraid of offending anyone"

topic = "us-canada geopolitical relations"

response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    reasoning={
        "effort": "high",
        #"generate_summary" : "concise" # detailed
        },
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}"
        }
    ]
)

In [38]:
print(response.output_text)

Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!


In [49]:
input_tokens = response.usage.input_tokens
output_tokens = response.usage.output_tokens
reasoning_tokens = response.usage.output_tokens_details.reasoning_tokens
net_output_tokens = output_tokens - reasoning_tokens

print(f'Input tokens: {input_tokens}')
print(f'Reasoning tokens: {reasoning_tokens}')
print(f'Net output tokens: {net_output_tokens}')
print(f'Output tokens: {output_tokens}')

Input tokens: 83
Reasoning tokens: 3264
Net output tokens: 52
Output tokens: 3316


In [51]:
response.reasoning

Reasoning(effort='high', generate_summary=None)

In [52]:
response.output

[ResponseReasoningItem(id='rs_67d1cdee27a08192beb956158fb8a02d04bbb1bb73a141e7', summary=[], type='reasoning', status=None),
 ResponseOutputMessage(id='msg_67d1cdf354348192a050188302163acc04bbb1bb73a141e7', content=[ResponseOutputText(annotations=[], text='Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!', type='output_text')], role='assistant', status='completed', type='message')]

In [55]:
response.output[-1].content[0].text

'Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!'

Structured output

The direct BaseModel passing is still in beta, so will convert class to JSON schema

https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses&example=structured-data

In [84]:
class Joke(BaseModel):
    """A joke about the user's preferred topic"""

    joke: str = Field(description="A joke about the user's preferred topic")
    explanation: str = Field(description="An explanation of the joke")

    @classmethod
    def get_name(cls):
        return "joke"

In [87]:
structured_format = {
    "format": {
        "type": "json_schema",
        "name": Joke.get_name(),
        "schema": {
            **Joke.model_json_schema(),
            "additionalProperties": False
        },
        "strict": True
    }
}

In [ ]:
response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    reasoning={
        "effort": "high",
        #"generate_summary" : "concise" # detailed
        },
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}"
        }
    ],
    text=structured_format
)

In [94]:
try:
    response_dict = json.loads(response.output[-1].content[0].text)
except json.JSONDecodeError as e:
    response_dict = response.output[-1].content[0].text

print(response_dict)

{'joke': "At the latest summit, the US declared, 'We need a wall to keep out our problems,' while Canada just smiled and said, 'Sorry, eh? How about we resolve them over some hockey and maple syrup instead?'", 'explanation': 'This joke plays on the classic stereotypes: the US with its bold, sometimes confrontational policies (like building walls), and Canada with its trademark politeness and laid-back charm, suggesting that even geopolitical disputes could be smoothed over with a friendly game of hockey and a taste of maple syrup.'}


#### Web search tool

https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses

Available values for `search_context_size`:

- high: Most comprehensive context, highest cost, slower response.
- medium (default): Balanced context, cost, and latency.
- low: Least context, lowest cost, fastest response, but potentially lower answer quality.

Bots: https://platform.openai.com/docs/bots

In [105]:
MODEL_NAME = 'gpt-4o'

response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}. Make sure your knowledge is up to date about the topic and cite your sources!"
        }
    ],
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
)

In [106]:
response.output

[ResponseFunctionWebSearch(id='ws_67d2cbd4a52c8192a0f61e3ac6b8e4a50c3c7808f881c0ef', status='completed', type='web_search_call'),
 ResponseOutputMessage(id='msg_67d2cbd81fd48192a4220fa8f2dcdc760c3c7808f881c0ef', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=319, start_index=226, title="Canada's incoming prime minister says he'll meet Trump if Canadian sovereignty is respected", type='url_citation', url='https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai')], text="Why did the United States propose making Canada the 51st state?\n\nBecause they figured, if you're going to impose tariffs on your neighbor's steel and aluminum, you might as well go all the way and annex their maple syrup too! ([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))\n\n*Note: This joke references recent tensions between the U.S. and Canada over tariffs and President Trump's remarks about Canada becoming the 51st state.*

In [107]:
response.output[0].model_dump()

{'id': 'ws_67d2cbd4a52c8192a0f61e3ac6b8e4a50c3c7808f881c0ef',
 'status': 'completed',
 'type': 'web_search_call'}

In [108]:
response.output[-1].model_dump()

{'id': 'msg_67d2cbd81fd48192a4220fa8f2dcdc760c3c7808f881c0ef',
 'content': [{'annotations': [{'end_index': 319,
     'start_index': 226,
     'title': "Canada's incoming prime minister says he'll meet Trump if Canadian sovereignty is respected",
     'type': 'url_citation',
     'url': 'https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai'}],
   'text': "Why did the United States propose making Canada the 51st state?\n\nBecause they figured, if you're going to impose tariffs on your neighbor's steel and aluminum, you might as well go all the way and annex their maple syrup too! ([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))\n\n*Note: This joke references recent tensions between the U.S. and Canada over tariffs and President Trump's remarks about Canada becoming the 51st state.* ",
   'type': 'output_text'}],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

In [109]:
### save output

output = {
   "id":"msg_67d2cbd81fd48192a4220fa8f2dcdc760c3c7808f881c0ef",
   "content":[
      {
         "annotations":[
            {
               "end_index":319,
               "start_index":226,
               "title":"Canada's incoming prime minister says he'll meet Trump if Canadian sovereignty is respected",
               "type":"url_citation",
               "url":"https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai"
            }
         ],
         "text":"Why did the United States propose making Canada the 51st state?\n\nBecause they figured, if you're going to impose tariffs on your neighbor's steel and aluminum, you might as well go all the way and annex their maple syrup too! ([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))\n\n*Note: This joke references recent tensions between the U.S. and Canada over tariffs and President Trump's remarks about Canada becoming the 51st state.* ",
         "type":"output_text"
      }
   ],
   "role":"assistant",
   "status":"completed",
   "type":"message"
}

In [111]:
output['content'][0]

{'annotations': [{'end_index': 319,
   'start_index': 226,
   'title': "Canada's incoming prime minister says he'll meet Trump if Canadian sovereignty is respected",
   'type': 'url_citation',
   'url': 'https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai'}],
 'text': "Why did the United States propose making Canada the 51st state?\n\nBecause they figured, if you're going to impose tariffs on your neighbor's steel and aluminum, you might as well go all the way and annex their maple syrup too! ([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))\n\n*Note: This joke references recent tensions between the U.S. and Canada over tariffs and President Trump's remarks about Canada becoming the 51st state.* ",
 'type': 'output_text'}

In [113]:
print(output['content'][0]['text'])

Why did the United States propose making Canada the 51st state?

Because they figured, if you're going to impose tariffs on your neighbor's steel and aluminum, you might as well go all the way and annex their maple syrup too! ([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))

*Note: This joke references recent tensions between the U.S. and Canada over tariffs and President Trump's remarks about Canada becoming the 51st state.* 


In [114]:
print(output['content'][0]['text'][226:319])

([apnews.com](https://apnews.com/article/413e1d8ab73b059f6bf92aa79e40d5a2?utm_source=openai))


News search and summarization

In [122]:
MODEL_NAME = 'gpt-4o'

sys = "Your job is to keep users up to date with the US stock market news. You are a professional market analyst."

usr = "The US stock market has been dropping for a while. I'd like to know the underlying reasoning behind and for how many more weeks / months we expect downward trends."

response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    instructions=sys,
    input=[
        {
            "role": "user",
            "content": usr
        }
    ],
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "high",
    }],
)

In [123]:
response.output

[ResponseFunctionWebSearch(id='ws_67d2d0e6f7188192852eacf5cd1fc6db0a06afc8514ea352', status='completed', type='web_search_call'),
 ResponseOutputMessage(id='msg_67d2d0e9c2bc8192aad8186da12afb940a06afc8514ea352', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=601, start_index=452, title='Corporate gloom deepens as new Trump tariffs take effect', type='url_citation', url='https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai'), AnnotationURLCitation(end_index=1070, start_index=916, title='Futures slip as government funding bill deadline looms, growth fears persist', type='url_citation', url='https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai'), AnnotationURLCitation(end_index=1600, start_index=1451, title='Corporate gloom deepens as new Trump tariffs take effect', type='url_citation', url='https://w

In [129]:
response.output[1].content[0].model_dump()

{'annotations': [{'end_index': 601,
   'start_index': 452,
   'title': 'Corporate gloom deepens as new Trump tariffs take effect',
   'type': 'url_citation',
   'url': 'https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai'},
  {'end_index': 1070,
   'start_index': 916,
   'title': 'Futures slip as government funding bill deadline looms, growth fears persist',
   'type': 'url_citation',
   'url': 'https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai'},
  {'end_index': 1600,
   'start_index': 1451,
   'title': 'Corporate gloom deepens as new Trump tariffs take effect',
   'type': 'url_citation',
   'url': 'https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai'},
  {'end_index': 2139,
   'start_index': 1902,
   'title': 'Ominous market signals show 

In [130]:
response.output[1].content[0].model_dump()['annotations']

[{'end_index': 601,
  'start_index': 452,
  'title': 'Corporate gloom deepens as new Trump tariffs take effect',
  'type': 'url_citation',
  'url': 'https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai'},
 {'end_index': 1070,
  'start_index': 916,
  'title': 'Futures slip as government funding bill deadline looms, growth fears persist',
  'type': 'url_citation',
  'url': 'https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai'},
 {'end_index': 1600,
  'start_index': 1451,
  'title': 'Corporate gloom deepens as new Trump tariffs take effect',
  'type': 'url_citation',
  'url': 'https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai'},
 {'end_index': 2139,
  'start_index': 1902,
  'title': 'Ominous market signals show more trouble could await US stock

In [134]:
def count_unique_titles(citations_list):
    # Extract all titles
    titles = [citation['title'] for citation in citations_list if 'title' in citation]
    
    # Count unique titles
    unique_titles = set(titles)
    unique_count = len(unique_titles)
    
    # Create a dictionary with counts and URLs for each title
    title_info = {}
    for citation in citations_list:
        if 'title' not in citation:
            continue
            
        title = citation['title']
        url = citation.get('url', 'No URL provided')
        
        if title not in title_info:
            title_info[title] = {'count': 1, 'urls': [url]}
        else:
            title_info[title]['count'] += 1
            if url not in title_info[title]['urls']:
                title_info[title]['urls'].append(url)
    
    return unique_count, title_info

In [133]:
unique_count, title_info = count_unique_titles(response.output[1].content[0].model_dump()['annotations'])

print(f"Number of unique titles: {unique_count}")
print("\nTitle information:")
for title, info in title_info.items():
    print(f"\n- '{title}'")
    print(f"  Count: {info['count']}")
    print(f"  URLs:")
    for url in info['urls']:
        print(f"    * {url}")

Number of unique titles: 6

Title information:

- 'Corporate gloom deepens as new Trump tariffs take effect'
  Count: 3
  URLs:
    * https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai

- 'Futures slip as government funding bill deadline looms, growth fears persist'
  Count: 2
  URLs:
    * https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai

- 'Ominous market signals show more trouble could await US stocks'
  Count: 2
  URLs:
    * https://www.reuters.com/markets/us/ominous-market-signals-show-more-trouble-could-await-us-stocks-2025-03-12/?utm_source=openai

- 'Introducing the 'Maleficent 7''
  Count: 1
  URLs:
    * https://www.ft.com/content/d8772ea6-ce65-47ba-bd3b-31758cf7c135?utm_source=openai

- 'Goldman Sachs lowers S&P 500 year-end target to 6,200'
  Count: 1
  URLs:
    * https://www.reuters.com/markets/us/goldm

In [138]:
display(Markdown(response.output[1].content[0].model_dump()['text']))

The recent downturn in the U.S. stock market can be attributed to several key factors:

1. **Trade Policies and Tariffs**: President Donald Trump's administration has implemented new tariffs on steel and aluminum imports, leading to increased costs for businesses and concerns about inflation. This has particularly affected industries like automotive and manufacturing, with companies such as Porsche considering price hikes to offset tariff impacts. ([reuters.com](https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai))

2. **Government Funding Uncertainty**: Investors are apprehensive about a potential government shutdown due to delays in passing a funding bill. The Senate is currently focused on a stopgap measure to prevent a partial shutdown, which could be extended until April 11. This uncertainty adds to market volatility. ([reuters.com](https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai))

3. **Economic Indicators and Recession Fears**: Despite a slight easing in inflation, the escalation of trade wars and policy unpredictability have heightened fears of an economic slowdown or recession. Analysts have noted that continuous geopolitical shifts make long-term business planning challenging, with over 900 major U.S. companies discussing tariffs in future outlooks. ([reuters.com](https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai))

4. **Market Technicals and Valuations**: The S&P 500 has fallen below critical trend lines, and major stocks like Nvidia and Tesla have seen significant declines. The "Magnificent 7" tech stocks have been rebranded as the "Maleficent 7" due to their substantial contributions to the market downturn. ([reuters.com](https://www.reuters.com/markets/us/ominous-market-signals-show-more-trouble-could-await-us-stocks-2025-03-12/?utm_source=openai), [ft.com](https://www.ft.com/content/d8772ea6-ce65-47ba-bd3b-31758cf7c135?utm_source=openai))

As for the duration of this downward trend, forecasts vary:

- **Goldman Sachs** has reduced its year-end target for the S&P 500 Index in 2025 from 6,500 to 6,200, citing increased policy uncertainty and concerns over economic growth. ([reuters.com](https://www.reuters.com/markets/us/goldman-sachs-cuts-sp-500-2025-year-end-target-6200-2025-03-12/?utm_source=openai))

- **Stifel** anticipates that the S&P 500 will peak in early 2025 before declining by 10% to 15% in the latter half of the year, attributing this forecast to slowed economic growth and persistent inflation. ([reuters.com](https://www.reuters.com/markets/us/stifel-expects-sp-500-peak-early-2025-before-falling-2024-12-12/?utm_source=openai))

Given these projections, the market may experience continued volatility over the next several months. However, it's important to note that market conditions are influenced by a complex interplay of factors, and predictions can change as new information becomes available.


## Recent Developments in U.S. Stock Market and Economic Policies:
- [Corporate gloom deepens as new Trump tariffs take effect](https://www.reuters.com/business/retail-consumer/corporate-gloom-deepens-new-trump-tariffs-take-effect-2025-03-12/?utm_source=openai)
- [Futures slip as government funding bill deadline looms, growth fears persist](https://www.reuters.com/markets/us/futures-slip-government-funding-bill-deadline-looms-growth-fears-persist-2025-03-13/?utm_source=openai)
- [Ominous market signals show more trouble could await US stocks](https://www.reuters.com/markets/us/ominous-market-signals-show-more-trouble-could-await-us-stocks-2025-03-12/?utm_source=openai) 

Web search with the old Chat.Completions API

Using `gpt-4o-search-preview` model

In [144]:
sys = "Your job is to keep users up to date with the US stock market news. You are a professional market analyst."

usr = "The US stock market has been dropping for a while. I'd like to know the underlying reasoning behind and for how many more weeks / months we expect downward trends."

r = client.chat.completions.create(
    model="gpt-4o-search-preview",
    messages=[
        {
            "role": "system",
            "content": sys
        },
        {
            "role": "user",
            "content": usr
        }
    ],
)

In [152]:
display(Markdown(r.choices[0].message.content))

The recent downturn in the U.S. stock market can be attributed to several interrelated factors:

1. **Trade Policies and Tariffs**: President Donald Trump's implementation of tariffs on European Union goods has intensified global trade tensions, leading to concerns about potential economic slowdowns. ([reuters.com](https://www.reuters.com/markets/global-markets-trading-day-2025-03-12/?utm_source=openai))

2. **Investor Shift to Value Stocks**: Amid market volatility, investors are reallocating funds from growth-oriented stocks, particularly in the technology sector, to value stocks in sectors like banks, energy, and utilities. This shift reflects a move towards stability in uncertain times. ([reuters.com](https://www.reuters.com/markets/us/us-investors-seek-refuge-value-funds-2025-03-12/?utm_source=openai))

3. **Hedge Fund De-risking**: Hedge funds have been reducing their equity positions and leverage to mitigate losses during increased market volatility. This de-risking has contributed to downward pressure on stock prices. ([ft.com](https://www.ft.com/content/a642cc9d-5c7c-4254-b6c1-be0bdc6fb4f1?utm_source=openai))

4. **Economic Indicators**: Recent data, including an uptick in the U.S. unemployment rate and jobless claims, has renewed fears of a potential recession. Such indicators can negatively impact investor confidence and market performance. ([cbsnews.com](https://www.cbsnews.com/news/why-is-the-stock-market-down-what-should-investors-do/?utm_source=openai))

As of March 13, 2025, major indices reflect these trends:

- **S&P 500 (SPY)**: Currently at 558.87 USD, up 0.55% from the previous close.

- **Dow Jones Industrial Average (DIA)**: At 413.95 USD, down 0.20% from the previous close.

- **Nasdaq Composite (QQQ)**: At 476.92 USD, up 1.16% from the previous close.

Predicting the exact duration of market downturns is challenging due to the complex interplay of economic policies, global events, and investor sentiment. While some analysts suggest that the market may experience continued volatility in the coming weeks, others believe that the recent de-risking by hedge funds and shifts towards value stocks could stabilize the market sooner. It's essential to monitor ongoing economic indicators and policy developments to gauge the market's trajectory.


## Recent Developments in U.S. Stock Market:
- [US investors seek refuge in value funds](https://www.reuters.com/markets/us/us-investors-seek-refuge-value-funds-2025-03-12/?utm_source=openai)
- [Hedge funds slash bets as Trump's trade war causes 'a lot of pain'](https://www.ft.com/content/a642cc9d-5c7c-4254-b6c1-be0bdc6fb4f1?utm_source=openai)
- [Trading Day: Finally, a bounce on Wall Street. But for how long?](https://www.reuters.com/markets/global-markets-trading-day-2025-03-12/?utm_source=openai) 

In [160]:
r.choices[0].message.annotations

[Annotation(type='url_citation', url_citation=AnnotationURLCitation(end_index=407, start_index=302, title='Trading Day: Finally, a bounce on Wall Street. But for how long?', url='https://www.reuters.com/markets/global-markets-trading-day-2025-03-12/?utm_source=openai')),
 Annotation(type='url_citation', url_citation=AnnotationURLCitation(end_index=818, start_index=700, title='US investors seek refuge in value funds', url='https://www.reuters.com/markets/us/us-investors-seek-refuge-value-funds-2025-03-12/?utm_source=openai')),
 Annotation(type='url_citation', url_citation=AnnotationURLCitation(end_index=1135, start_index=1042, title="Hedge funds slash bets as Trump's trade war causes 'a lot of pain'", url='https://www.ft.com/content/a642cc9d-5c7c-4254-b6c1-be0bdc6fb4f1?utm_source=openai')),
 Annotation(type='url_citation', url_citation=AnnotationURLCitation(end_index=1492, start_index=1374, title='4 reasons the stock market is plunging — and what experts say you should do - CBS News', u

Additional tools

Custom tools: define functions and provide as JSON schemas

In [164]:
import requests

def get_weather(latitude: float, longitude: float):

    """Get current temperature for provided coordinates in celsius."""

    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    
    return data['current']['temperature_2m']

In [165]:
### test on Budapest

lat = '47.497913'
lon = '19.040236'

bp_weather = get_weather(lat, lon)
print(f'Weather data for Budapest is: {bp_weather} C')

Weather data for Budapest is: 12.4 C


In [169]:
import inspect

def function_to_json(func) -> dict:
    """
    Converts a Python function into a JSON-serializable dictionary
    that describes the function's signature for LLM function calling.
    """
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }
    
    signature = inspect.signature(func)
    parameters = {}
    
    for param in signature.parameters.values():
        param_type = type_map.get(param.annotation, "string")
        parameters[param.name] = {"type": param_type}
    
    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]
    
    return {
        "type": "function",
        "name": func.__name__,
        "description": func.__doc__ or "",
        "parameters": {
            "type": "object",
            "properties": parameters,
            "required": required,
            "additionalProperties": False
        },
        "strict": True
}

# Generate the schema
tool_schema = function_to_json(get_weather)
print(tool_schema)

{'type': 'function', 'name': 'get_weather', 'description': 'Get current temperature for provided coordinates in celsius.', 'parameters': {'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, 'strict': True}


In [ ]:
user_msg = 'What is the weather like in New York?'

response = client.responses.create(
    model='gpt-4o',
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    instructions="You are a helpful AI assistant",
    input=[
        {
            "role": "user",
            "content": user_msg
        }
    ],
    tools=[tool_schema],
)

response.output

[ResponseFunctionToolCall(id='fc_67d328ff8f188192b2585555471780480b90233318eabe11', arguments='{"latitude":40.7128,"longitude":-74.006}', call_id='call_zBRYrfNAZEllq8bb1FxeARRR', name='get_weather', type='function_call', status='completed')]

Coordinates make sense: https://www.latlong.net/c/?lat=40.7143&long=-74.0060

Now add more tools!

In [175]:
user_msg = 'First tell me the current NVIDIA stock price, then look up the weather in New York!'

response = client.responses.create(
    model='gpt-4o',
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    instructions="You are a helpful AI assistant",
    input=[
        {
            "role": "user",
            "content": user_msg
        }
    ],
    tools=[
        tool_schema,
        {
            "type": "web_search_preview",
            "search_context_size": "high",
        }
        ],
)


response.output

[ResponseFunctionWebSearch(id='ws_67d32a06fce48192bc3e8e2bd8806de90d396eb2505debc1', status='completed', type='web_search_call'),
 ResponseOutputMessage(id='msg_67d32a0a0a4c81928f04084d36ec50aa0d396eb2505debc1', content=[ResponseOutputText(annotations=[], text='As of March 13, 2025, NVIDIA Corporation (NVDA) stock is trading at $117.35, up $1.61 (1.39%) from the previous close.\n\n## Stock market information for NVIDIA Corp (NVDA)\n- NVIDIA Corp is a equity in the USA market.\n- The price is 117.35 USD currently with a change of 1.61 USD (0.01%) from the previous close.\n- The latest open price was 116.99 USD and the intraday volume is 220606315.\n- The intraday high is 117.73 USD and the intraday low is 113.85 USD.\n- The latest trade time is Thursday, March 13, 18:39:37 UTC.\n\n\nIn New York, NY, the current weather is mostly cloudy with a temperature of 48°F (9°C).\n\n## Weather for New York, NY:\nCurrent Conditions: Mostly cloudy, 48°F (9°C)\n\nDaily Forecast:\n* Thursday, March 13

First, it called the web search, it also ran the 'function', as it should, then it called my function and is waiting for me to run it!

Takeaways:

General:

- system_message can be passed as `instructions` parameter or as a separate `system` message in the `input` parameter.
- there is a `generate_summary` parameter that could be used to show concise or detailed reasoning - but does not work yet with any of the reasoning models.
- no direct Structured Output support, but `json_schema` in `text` field works

Web search:
- for non-obvious web queries, ask in the prompt the model to cite its sources otherwise the `annotations` is empty
- only `gpt-4o` and `gpt-4.5` support - does not run with o1 or o3-mini
- `gpt-4.5-preview` finished in 7 minutes but with incomplete status
- for market news search the used web sources are high quality (ft, reuters)
- `gpt-4o` + web search tool in Responses API ~ `gpt-4o-search-preview` in Chat.Competions API